# Comparing 04/05 to 06/07 vs 16/17 to 18/19 Shot Tracking



# How to

Check out the guide in ParseShotData for how to get this data and parse it. 

Also check this out for the shot aggregations

https://github.com/rd11490/NBA_Tutorials/blob/master/shot_bins/binning_shots.py

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt  

cwd = 'G:\\NBA'
#cwd = os.getcwd()

In [2]:
def loadPcl(cwd, sub, fname, mode='rb'):
    if sub == "":
        with open(os.path.join(cwd,fname ), mode) as f: 
            df = pickle.load(f)
    else:
        with open(os.path.join(cwd,sub,fname ), mode) as f: 
            df = pickle.load(f)
    return df
def writeDF(df,cwd, sub, fname):
    if sub == "":
        dir = os.path.join( cwd,fname)

    else:
        dir = os.path.join( cwd,sub,fname)

    df.to_pickle(dir)
    print("wrote " + fname)

In [3]:
reg0405 = loadPcl(cwd,"shotData","shotsReg0405.pkl")
reg0506 = loadPcl(cwd,"shotData","shotsReg0506.pkl")
reg0607 = loadPcl(cwd,"shotData","shotsReg0607.pkl")

reg1617 = loadPcl(cwd,"shotData","shotsReg1617.pkl")
reg1718 = loadPcl(cwd,"shotData","shotsReg1718.pkl")
reg1819 = loadPcl(cwd,"shotData","shotsReg1819.pkl")

comb = reg1617.append(reg1718).append(reg1819)
combOld = reg0405.append(reg0506).append(reg0607)

df =reg1819

c:\program files\python37\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
def agg_shots(group):
    total_shots = group['FGA'].sum() # Calculate the number of attempted shots
    made_shots = group["FGM"].sum() # Calculate the number of made shots
    percentage = round(made_shots/total_shots, 3) # Calculate the field goal percentage (round to 3 decimal places)
    
    shotpts = group["SHOT_PTS"].sum() # 0 for miss, 2 for 2pt, 3 for 3pt
    pps = round(shotpts/total_shots, 3)

    return pd.Series([made_shots, total_shots, percentage, pps]) 

In [5]:
def makeTable(df,grpByCol):
    agg = df.groupby(grpByCol).apply(agg_shots).reset_index()
    agg.columns = (grpByCol + [ "FGM", "FGA", "FG%","PPS"])
    agg = agg.sort_values("FGA",ascending=False)
    
    return agg

In [6]:
def makeTables(df):
    aggType,aggZone, aggSvSt,aggZT,aggPVT, aggPZ, aggPZT, aggPV,aggPVZT,aggTF = 0,0,0,0,0,0,0,0,0,0
    
    aggType = makeTable(df,["SHOT_TYPE"])
    aggZone = makeTable(df,["SHOT_ZONE"])
    #aggSvSt = makeTable(df,["SHOT_VALUE","SHOT_TYPE"])
    aggZT = makeTable(df,["SHOT_ZONE","SHOT_TYPE"])
    
    aggPVT = makeTable(df,["PLAYER_NAME","SHOT_VALUE","SHOT_TYPE"])
    aggPZ = makeTable(df,["PLAYER_NAME","SHOT_ZONE"])
    aggPZT = makeTable(df,["PLAYER_NAME","SHOT_ZONE","SHOT_TYPE"])
    aggPV = makeTable(df,["PLAYER_NAME","SHOT_VALUE"])
    #aggPVZT = makeTable(df,["PLAYER_NAME","SHOT_VALUE","SHOT_ZONE","SHOT_TYPE"])
    
    df2= df[df.SHOT_TYPE.isin(['Turnaround Jump Shot',"Fadeaway Jump Shot","Turnaround Fadeaway shot","Turnaround Fadeaway Bank Jump Shot"])]
    aggTF = makeTable(df2,["PLAYER_NAME","SHOT_ZONE"])

    return aggType,aggZone, aggSvSt,aggZT,aggPVT, aggPZ, aggPZT, aggPV,aggPVZT,aggTF

In [7]:
cols = ["Jump Shot","Pullup Jump shot","Step Back Jump shot",
        'Turnaround Jump Shot',"Fadeaway Jump Shot","Turnaround Fadeaway shot","Turnaround Fadeaway Bank Jump Shot"]#'Running Jump Shot', 'Running Pull-Up Jump Shot',

In [ ]:
# league 1819
#aggType,aggZone, aggSvSt,aggZT,aggPVT, aggPZ, aggPZT, aggPV,aggPVZT,aggTF = makeTables(reg1819)
#aggType18,aggZone18, aggSvSt18,aggZT18,aggPVT18, aggPZ18, aggPZT18, aggPV18,aggPVZT18,aggTF18 = makeTables(reg1718)

# 0506
#aggType06,aggZone06, aggSvSt06,aggZT06,aggPVT06, aggPZ06, aggPZT06, aggPV06, aggPVZT06,aggTF06 = makeTables(reg0506)

In [8]:
aggType,aggZone, aggSvSt,aggZT,aggPVT, aggPZ, aggPZT, aggPV,aggPVZT,aggTF = makeTables(comb)

In [9]:
aggType06,aggZone06, aggSvSt06,aggZT06,aggPVT06, aggPZ06, aggPZT06, aggPV06,aggPVZT06,aggTF06 = makeTables(combOld)


# 16/17 to 18/19


In [10]:
aggZT[(aggZT.SHOT_ZONE=="MR")].head(10)

,SHOT_ZONE,SHOT_TYPE,FGM,FGA,FG%,PPS
64,MR,Jump Shot,18726.0,53466.0,0.350,0.702
68,MR,Pullup Jump shot,15254.0,33177.0,0.460,0.920
77,MR,Step Back Jump shot,4332.0,9688.0,0.447,0.895
58,MR,Fadeaway Jump Shot,2318.0,5676.0,0.408,0.817
84,MR,Turnaround Jump Shot,1724.0,4369.0,0.395,0.789
82,MR,Turnaround Fadeaway shot,1713.0,3844.0,0.446,0.891
60,MR,Floating Jump shot,898.0,2289.0,0.392,0.785
51,MR,Driving Floating Jump Shot,769.0,1893.0,0.406,0.812
63,MR,Jump Bank Shot,530.0,1112.0,0.477,0.953
50,MR,Driving Floating Bank Jump Shot,437.0,824.0,0.530,1.061


In [11]:
aggZone

,SHOT_ZONE,FGM,FGA,FG%,PPS
5,RA,128875.0,206591.0,0.624,1.248
0,ATB3,58290.0,165477.0,0.352,1.057
3,MR,48422.0,120186.0,0.403,0.806
4,PAINT,39831.0,98188.0,0.406,0.811
2,LC3,9754.0,25312.0,0.385,1.156
6,RC3,9286.0,23851.0,0.389,1.168
1,BCOURT,29.0,1489.0,0.019,0.058


In [12]:
tf = aggTF[(aggTF.SHOT_ZONE =="MR") ]

In [13]:
js = aggPZT[(aggPZT.SHOT_ZONE =="MR") & (aggPZT.SHOT_TYPE == "Jump Shot")  ]

In [14]:
mrg = pd.merge(tf, js, left_on = ["PLAYER_NAME"], right_on=["PLAYER_NAME"], how ="left", suffixes=('', '_JS'))
mrg["FG%_DIFF"] = mrg["FG%"] - mrg["FG%_JS"]

mrg[["PLAYER_NAME","FGM","FGA","FG%","FG%_JS","FG%_DIFF","FGM_JS","FGA_JS"]].head(10)#.sort_values("FG%_DIFF",ascending=False)#.head(15)
#mrg.head()

,PLAYER_NAME,FGM,FGA,FG%,FG%_JS,FG%_DIFF,FGM_JS,FGA_JS
0,LaMarcus Aldridge,263.0,599.0,0.439,0.426,0.013,362.0,849.0
1,DeMar DeRozan,152.0,364.0,0.418,0.344,0.074,162.0,471.0
2,Dwyane Wade,112.0,288.0,0.389,0.249,0.140,69.0,277.0
3,Kevin Durant,158.0,280.0,0.564,0.403,0.161,160.0,397.0
4,LeBron James,113.0,273.0,0.414,0.367,0.047,87.0,237.0
5,Andrew Wiggins,104.0,267.0,0.390,0.271,0.119,107.0,395.0
6,Dirk Nowitzki,92.0,223.0,0.413,0.454,-0.041,248.0,546.0
7,Marc Gasol,96.0,221.0,0.434,0.434,0.000,247.0,569.0
8,Klay Thompson,111.0,206.0,0.539,0.380,0.159,217.0,571.0
9,Harrison Barnes,90.0,200.0,0.450,0.366,0.084,164.0,448.0


# 04/05 to 06/07


In [15]:
aggZT06[(aggZT06.SHOT_ZONE=="MR")].head(10)

,SHOT_ZONE,SHOT_TYPE,FGM,FGA,FG%,PPS
33,MR,Jump Shot,68177.0,180649.0,0.377,0.755
43,MR,Turnaround Jump Shot,3888.0,7357.0,0.528,1.057
27,MR,Fadeaway Jump Shot,3295.0,5612.0,0.587,1.174
31,MR,Jump Bank Shot,2082.0,3170.0,0.657,1.314
38,MR,Running Jump Shot,1618.0,2884.0,0.561,1.122
30,MR,Hook Shot,393.0,1128.0,0.348,0.697
32,MR,Jump Hook Shot,257.0,493.0,0.521,1.043
42,MR,Turnaround Hook Shot,132.0,261.0,0.506,1.011
37,MR,Running Hook Shot,50.0,89.0,0.562,1.124
29,MR,Hook Bank Shot,23.0,28.0,0.821,1.643


In [16]:
aggZone06

,SHOT_ZONE,FGM,FGA,FG%,PPS
3,MR,79935.0,201719.0,0.396,0.793
5,RA,112985.0,190672.0,0.593,1.185
0,ATB3,29736.0,84800.0,0.351,1.052
4,PAINT,30824.0,76078.0,0.405,0.810
2,LC3,6694.0,17505.0,0.382,1.147
6,RC3,6265.0,16128.0,0.388,1.164
1,BCOURT,22.0,1110.0,0.020,0.059


In [17]:
# turnarounds and fadeaways vs js
tf06 = aggTF06[(aggTF06.SHOT_ZONE =="MR") ]#.head(10)
js06 = aggPZT06[(aggPZT06.SHOT_ZONE =="MR") & (aggPZT06.SHOT_TYPE == "Jump Shot")  ]#.head(10)

mrg06 = pd.merge(tf06, js06, left_on = ["PLAYER_NAME"], right_on=["PLAYER_NAME"], how ="left", suffixes=('', '_JS'))
mrg06["FG%_DIFF"] = mrg06["FG%"] - mrg06["FG%_JS"]
#mrg.sort_values("FG%_DIFF",ascending=False)#.head(15)
mrg06[["PLAYER_NAME","FGM","FGA","FG%","FG%_JS","FG%_DIFF","FGM_JS","FGA_JS"]].head(15)#.sort_values("FG%_DIFF",ascending=False)#.head(15)
#mrg.head()

,PLAYER_NAME,FGM,FGA,FG%,FG%_JS,FG%_DIFF,FGM_JS,FGA_JS
0,Kevin Garnett,242.0,421.0,0.575,0.414,0.161,624.0,1507.0
1,Dirk Nowitzki,236.0,385.0,0.613,0.430,0.183,811.0,1884.0
2,Yao Ming,189.0,338.0,0.559,0.308,0.251,130.0,422.0
3,Kobe Bryant,204.0,305.0,0.669,0.347,0.322,600.0,1729.0
4,LeBron James,115.0,241.0,0.477,0.335,0.142,497.0,1483.0
5,Jason Richardson,111.0,238.0,0.466,0.317,0.149,231.0,729.0
6,Vince Carter,117.0,228.0,0.513,0.383,0.130,442.0,1155.0
7,Rasheed Wallace,122.0,226.0,0.540,0.370,0.170,298.0,805.0
8,Tracy McGrady,107.0,211.0,0.507,0.409,0.098,685.0,1675.0
9,Jermaine O'Neal,152.0,208.0,0.731,0.308,0.423,331.0,1076.0


# 3P Shot Types 16/17 to 18/19

In [18]:
mrg3p = pd.merge(aggPVT, aggPV[["PLAYER_NAME","SHOT_VALUE","FGA","FG%"]], left_on = ["PLAYER_NAME","SHOT_VALUE"], right_on=["PLAYER_NAME","SHOT_VALUE"], how ="left", suffixes=('', '_3P'))
mrg3p["FG%_DIFF"] = mrg3p["FG%"] - mrg3p["FG%_3P"]
#mrg.head()

In [21]:
p3 = mrg3p[(mrg3p.SHOT_VALUE==3) & ((mrg3p.SHOT_TYPE=="Step Back Jump shot") | (mrg3p.SHOT_TYPE=="Pullup Jump shot"))& (mrg3p.FGA>70) ] #& (aggPS.FGA>30)& (aggPS["FG%"]>0.42)
p3[["PLAYER_NAME","SHOT_VALUE","SHOT_TYPE","FGM","FGA","FG%","FG%_3P","FG%_DIFF","FGA_3P"]].sort_values("FG%_DIFF",ascending=False).head(20)

,PLAYER_NAME,SHOT_VALUE,SHOT_TYPE,FGM,FGA,FG%,FG%_3P,FG%_DIFF,FGA_3P
1843,Andre Iguodala,3,Pullup Jump shot,48.0,75.0,0.640,0.331,0.309,438.0
1264,Draymond Green,3,Pullup Jump shot,60.0,106.0,0.566,0.300,0.266,684.0
1776,Quinn Cook,3,Pullup Jump shot,50.0,78.0,0.641,0.418,0.223,330.0
523,Klay Thompson,3,Pullup Jump shot,153.0,242.0,0.632,0.418,0.214,1766.0
332,Stephen Curry,3,Pullup Jump shot,212.0,341.0,0.622,0.424,0.198,2099.0
820,Stephen Curry,3,Step Back Jump shot,96.0,159.0,0.604,0.424,0.180,2099.0
1696,Austin Rivers,3,Pullup Jump shot,41.0,82.0,0.500,0.357,0.143,984.0
1593,Wesley Matthews,3,Step Back Jump shot,44.0,87.0,0.506,0.371,0.135,1284.0
503,Kevin Durant,3,Pullup Jump shot,127.0,250.0,0.508,0.384,0.124,1113.0
807,Lou Williams,3,Pullup Jump shot,78.0,162.0,0.481,0.362,0.119,1254.0


In [23]:
#p3[["PLAYER_NAME","SHOT_VALUE","SHOT_TYPE","FGM","FGA","FG%","FG%_3P","FG%_DIFF","FGA_3P"]].head(20)